In [21]:
# import pdb; pdb.set_trace()
# import pudb; pudb.set_trace()

# df.style.background_gradient(
#     cmap='coolwarm', 
#     axis=None, 
#     vmin=-1, 
#     vmax=1
# )  # 'Blues', 'coolwarm', 'RdBu_r' & 'BrBG'

# df.style.set_sticky(axis="index")

# pd.set_option('precision', 2)

# from pandas_profiling import ProfileReport
# profile = ProfileReport(df, title="Pandas Profiling Report", explorative=True)

# .plot_bokeh(sizing_mode="scale_height")
# .plot_bokeh(kind='hist', bins=1000, sizing_mode="scale_height", xlim=[0, .1])

# Imports

In [51]:
%load_ext autoreload
%autoreload 2

import pandas as pd
pd.set_option('float_format', '{:f}'.format)
pd.set_option('display.max_rows', 40)

pd.set_option('display.max_columns', 99)

import matplotlib as mpl
from matplotlib import style
style.use('fivethirtyeight')
mpl.rcParams['figure.figsize'] = [20, 7]

import datetime as dt
import pickle
import numpy as np
import pandas as pd
import pandas_bokeh
pandas_bokeh.output_file("tmp/bokeh_output.html")
import ray

from data_layer import storage_adaptor, fsspec_factory, data_access, arrow_dataset
from tick_filter import streaming_tick_filter
from tick_sampler import streaming_tick_sampler, daily_stats
from utilities import date_fu, project_globals as g
from workflows import sampler_task, sampler_flow, syncbar_flow, syncbar_task
from workflows.configs import renko_v1, renko_v2, renko_v3
from sample_features import stacked
from filters import jma

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
config = renko_v2.config

prefix_data = '/data/trades'

prefix_meta = f"/bars/{config['meta']['config_id']}/meta"

prefix_df = f"/bars/{config['meta']['config_id']}/df/symbol=GLD"

prefix_sb = f"/bars/{config['meta']['config_id']}/sync_bars/clock_symbol=SAND/symbol=GLD"

In [ ]:
df = data_access.fetch_sd_data('AEM', prefix=prefix_df, date='2019-01-03')

df

# load multi-file dataset

In [24]:
prefix_df

'/bars/renko_v2/df/symbol=GLD'

In [25]:
ds = arrow_dataset.get_dataset(prefix=prefix_df)

In [26]:
df = ds.to_table().to_pandas()

# JMA filter

In [7]:
def plot_jma(df: pd.DataFrame, col: str, winlen: int, plot_cols: list=[], 
             fill_gap: bool=False, expand: bool=False) -> pd.DataFrame:

    df = df.copy()
    if fill_gap:
        df = stacked.fill_gaps_dates(df, col)

    df = jma.jma_filter_df(df, col, winlen, expand=expand)
    df[[col, f"{col}_jma{winlen}"]+plot_cols].plot_bokeh(sizing_mode="scale_height")
    return df

In [50]:
out = plot_jma(df, 'price_vwap', winlen=200, plot_cols=[], fill_gap=True, expand=True)

In [9]:
# bar featureizer
def gap_aware_filter(df: pd.DataFrame, col: str, winlens: list[int], fill_gap: bool) -> pd.DataFrame:
    jdf = df.copy()
    if fill_gap:
        jdf = stacked.fill_gaps_dates(jdf, col)

    for winlen in winlens:
        if winlen > 99:
            jdf.loc[:, f"{col}_jma{winlen}"] = jma.jma_expanding_filter(
                jdf[col], winlen, power=1, phase=0
            )
        else:
            jdf.loc[:, f"{col}_jma{winlen}"] = jma.jma_rolling_filter(
                jdf[col], winlen, power=1, phase=0
            )
    
        jdf.loc[:, f"{col}_jma{winlen}_diff"] = jdf[f"{col}_jma{winlen}"].diff()
        jdf.loc[:, f"{col}_jma{winlen}_resid"] = jdf[col] - jdf[f"{col}_jma{winlen}"]
    
    jdf.loc[:, f"{col}_diff"] = jdf[f"{col}"].diff()  # add diff for original col
    
    return jdf[jdf.bar_trigger != 'gap_filler']


def filter_cols(df: pd.DataFrame, params: list[dict]) -> pd.DataFrame:
    _df = df.copy()
    for param in params:
        print(param)
        _df = gap_aware_filter(
            _df, 
            col=param['col'], 
            winlens=param['winlens'], 
            fill_gap=param['fill_gap'],
        )

    return _df

# bar features

In [48]:
# convert duration
df['duration_10s'] = round(df.duration_td.astype('int64')/10**9/10)

In [106]:
# get bar 'min in day'
df['min_in_day'] = (df.close_at.dt.hour * 60) + df.close_at.dt.minute

In [ ]:
# get bar 'day of week'
df['day_of_week'] = df.close_at.dt.dayofweek

In [53]:
# get weighted IQR
df['wiqr'] = df.price_wq95 - df.price_wq05

In [109]:
# add renko trend/reversal label
continue_mask = df.bar_trigger.shift(1) == df.bar_trigger
duration_mask = df.bar_trigger.shift(1) == 'duration'
df.loc[continue_mask, 'renko_type'] = 'renko_trend'
df.loc[~continue_mask, 'renko_type'] = 'renko_reversal'
df.loc[duration_mask, 'renko_type'] = 'duration'

df[['renko_reversal', 'renko_trend']] = pd.get_dummies(df.renko_type, drop_first=True)

In [111]:
plot_jma(df, 'price_return', 55)

,bar_trigger,open_at,close_at,duration_td,tick_count,volume,dollars,tick_imbalance,volume_imbalance,price_high,price_low,price_open,price_close,price_range,price_return,price_vwap,price_wq05,price_wq25,price_wq50,price_wq75,price_wq95,label_side,label_outcome,label_rrr,label_outcome_at,label_start_at,label_end_at,symbol,date,config_id,renko_size,duration_10s,wiqr,close_min_in_day,renko_type,price_return_jma55
0,renko_up,2019-01-02 09:30:00.131921-05:00,2019-01-02 09:34:46.704074-05:00,0 days 00:04:46.572153,983,334455,40588422.436127,129,176413,121.430000,121.310097,121.349998,121.430000,0.119904,0.080002,121.356901,121.339500,121.349998,121.349998,121.360001,121.400002,short,profit,-2.000000,2019-01-02 09:51:04.624883-05:00,2019-01-02 09:34:46.704074-05:00,2019-01-02 10:07:46.704074-05:00,GLD,2019-01-02,renko_v2,0.083822,29.000000,0.060501,574,renko_reversal,NaN
1,duration,2019-01-02 09:34:46.704176-05:00,2019-01-02 10:07:48.145198-05:00,0 days 00:33:01.441022,5100,1061484,128845780.745354,-230,-139828,121.480003,121.262497,121.430000,121.440002,0.217506,0.010002,121.382688,121.285004,121.330002,121.389999,121.435501,121.459999,long,profit,2.500000,2019-01-02 10:34:36.104698-05:00,2019-01-02 10:07:48.145198-05:00,2019-01-02 10:40:48.145198-05:00,GLD,2019-01-02,renko_v2,0.083822,198.000000,0.174995,607,renko_reversal,NaN
2,renko_up,2019-01-02 10:07:50.011431-05:00,2019-01-02 10:08:46.692614-05:00,0 days 00:00:56.681183,507,119142,14474032.090485,81,23112,121.529999,121.440002,121.440002,121.529999,0.089996,0.089996,121.485556,121.449997,121.470001,121.480003,121.510002,121.519997,neutral,neutral,0.000000,2019-01-02 10:34:16.619297-05:00,2019-01-02 10:08:46.692614-05:00,2019-01-02 10:41:46.692614-05:00,GLD,2019-01-02,renko_v2,0.083822,6.000000,0.070000,608,duration,NaN
3,renko_up,2019-01-02 10:08:46.692640-05:00,2019-01-02 10:15:31.824610-05:00,0 days 00:06:45.131970,1025,194440,23619217.213989,-57,-23746,121.620003,121.370003,121.529999,121.620003,0.250000,0.090004,121.473036,121.379997,121.415001,121.480003,121.519997,121.580002,neutral,neutral,0.000000,2019-01-02 10:35:40.518385-05:00,2019-01-02 10:15:31.824610-05:00,2019-01-02 10:48:31.824610-05:00,GLD,2019-01-02,renko_v2,0.083822,41.000000,0.200005,615,renko_trend,NaN
4,duration,2019-01-02 10:15:32.138740-05:00,2019-01-02 10:48:35.289868-05:00,0 days 00:33:03.151128,4163,781511,95013515.768280,63,26667,121.690002,121.470001,121.610001,121.620003,0.220001,0.010002,121.576684,121.500000,121.529999,121.550003,121.639999,121.669998,neutral,neutral,0.000000,2019-01-02 11:21:35.289868-05:00,2019-01-02 10:48:35.289868-05:00,2019-01-02 11:21:35.289868-05:00,GLD,2019-01-02,renko_v2,0.083822,198.000000,0.169998,648,renko_reversal,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17394,renko_up,2021-01-28 14:14:37.985949440-05:00,2021-01-28 14:26:56.701402112-05:00,0 days 00:12:18.715452672,978,124120,21458853.891617,138,41168,172.960007,172.780106,172.839996,172.960007,0.179901,0.120010,172.887962,172.809998,172.869995,172.899994,172.919998,172.929993,short,profit,-2.000000,2021-01-28 14:34:58.088518912-05:00,2021-01-28 14:26:56.701402112-05:00,2021-01-28 14:59:56.701402112-05:00,GLD,2021-01-28,renko_v2,0.121712,74.000000,0.119995,866,renko_trend,-0.035020
17395,renko_down,2021-01-28 14:27:01.013269248-05:00,2021-01-28 14:34:58.088523776-05:00,0 days 00:07:57.075254528,1462,188245,32542210.181442,-176,-6969,173.000000,172.710007,172.955002,172.710007,0.289993,-0.244995,172.871578,172.740005,172.789993,172.880005,172.960007,172.979996,long,profit,2.000000,2021-01-28 15:01:32.549214464-05:00,2021-01-28 14:34:58.088523776-05:00,2021-01-28 15:07:58.088523776-05:00,GLD,2021-01-28,renko_v2,0.121712,48.000000,0.239990,874,renko_reversal,-0.035627
17396,renko_up,2021-01-28 14:34:58.088523776-05:00,2021-01-28 15:01:41.155846656-05:00,0 days 00:26:43.067322880,3259,455799,78760666.122864,49,1

In [115]:
# filter & diff & resid bars/cols
params = [
    {'col': 'price_open', 'winlens': [5], 'fill_gap': True},
    {'col': 'price_low', 'winlens': [5], 'fill_gap': True},
    {'col': 'price_high', 'winlens': [5], 'fill_gap': True},
    {'col': 'price_close', 'winlens': [5], 'fill_gap': True},
    {'col': 'price_vwap', 'winlens': [5, 50, 200, 500], 'fill_gap': True},
    {'col': 'tick_count', 'winlens': [5], 'fill_gap': False},
    {'col': 'dollars', 'winlens': [5], 'fill_gap': False},
    {'col': 'volume_imbalance', 'winlens': [5], 'fill_gap': False},
    {'col': 'duration_10s', 'winlens': [5], 'fill_gap': False},
    {'col': 'wiqr', 'winlens': [5], 'fill_gap': False},
    {'col': 'duration_10s', 'winlens': [5], 'fill_gap': False},    
]

df2 = filter_cols(df, params)

{'col': 'price_low', 'winlens': [5], 'fill_gap': True}
{'col': 'price_high', 'winlens': [5], 'fill_gap': True}
{'col': 'price_close', 'winlens': [5], 'fill_gap': True}
{'col': 'price_vwap', 'winlens': [5, 50, 200, 500], 'fill_gap': True}
{'col': 'tick_count', 'winlens': [5], 'fill_gap': False}
{'col': 'dollars', 'winlens': [5], 'fill_gap': False}
{'col': 'volume_imbalance', 'winlens': [5], 'fill_gap': False}
{'col': 'duration_10s', 'winlens': [5], 'fill_gap': False}
{'col': 'wiqr', 'winlens': [5], 'fill_gap': False}
{'col': 'duration_10s', 'winlens': [5], 'fill_gap': False}


In [137]:
drop_cols = [
    'bar_trigger',
    'renko_type',
    'open_at',
    'close_at',
    'duration_td',
    'price_high',
    'price_low',
    'price_open',
    'price_close',
    'price_range',
    'price_return',
    'price_vwap',
    'price_wq05',
    'price_wq25',
    'price_wq50',
    'price_wq75',
    'price_wq95',
    'label_side',
    'label_outcome',
    'label_rrr',
    'label_outcome_at',
    'label_start_at',
    'label_end_at',
    'symbol',
    'date',
    'config_id',
    'renko_size',
    'price_low_jma5',
    'price_high_jma5',
    'price_close_jma5',
    'price_vwap_jma5',
    'price_vwap_jma50',
    'price_vwap_jma200',
    'price_vwap_jma500',
    'tick_count_jma5',
    'dollars_jma5',
    'volume_imbalance_jma5',
    'duration_10s_jma5',
    'wiqr_jma5',
    ]

In [128]:
df3 = df2.drop(columns=drop_cols)

In [132]:
pd.get_dummies(df3.renko_type, drop_first=True)

,renko_reversal,renko_trend
0,1,0
1,1,0
2,0,0
3,0,1
4,1,0
...,...,...
24718,0,1
24719,1,0
24720,1,0
24721,1,0


In [126]:
train.renko_type.value_counts()

renko_trend       9043
renko_reversal    5772
duration          2584
Name: renko_type, dtype: int64

# Ray setup

In [ ]:
ray.init(dashboard_host='0.0.0.0', dashboard_port=1111, ignore_reinit_error=True)

In [ ]:
ray.shutdown()

In [ ]:
all_symbols = data_access.list_sd_data(prefix='/bars/renko_v2/df', source='remote')

In [ ]:
large_caps, mid_caps = syncbar_task.segment_symbols(
    all_symbols, start_date='2019-01-01', end_date='2021-01-01'
)

In [ ]:
large_caps

In [ ]:
mid_caps

# sync bars

In [ ]:
config = {
    'config_id': 'renko_v2',
    'start_date': '2019-01-01',
    'end_date': '2021-01-01',
    'source': 'remote',
    'destination': 'both',
    'on_ray': True,
}

In [ ]:
syncbar_flow.run(config)

# Tick Sampler 

In [ ]:
bds = sampler_flow.run(config)

# fracdiff test

In [ ]:
import statsmodels.tsa.stattools as stattools
import fracdiff

X = bdf.price_close.values.reshape(-1, 1)

fs = fracdiff.FracdiffStat(mode="valid")

Xdiff = fs.fit_transform(X)

_, pvalue, _, _, _, _ = stattools.adfuller(Xdiff)

corr = np.corrcoef(X[-Xdiff.size :, 0], Xdiff.reshape(-1))[0][1]

print("* Order: {:.2f}".format(fs.d_[0]))
print("* ADF p-value: {:.2f} %".format(100 * pvalue))
print("* Correlation with the original time-series: {:.2f}".format(corr))

spx = bdf.price_close

import matplotlib.pyplot as plt

spx_diff = pd.Series(Xdiff.reshape(-1), index=spx.index[-Xdiff.size :])

fig, ax_s = plt.subplots(figsize=(24, 6))
plt.title("S&P 500 and its differentiation preserving memory")
ax_d = ax_s.twinx()

plot_s = ax_s.plot(spx, color="blue", linewidth=4, label="S&P 500 (left)")
plot_d = ax_d.plot(
    spx_diff,
    color="orange",
    linewidth=4,
    label=f"S&P 500, {fs.d_[0]:.2f} th diff (right)",
)
plots = plot_s + plot_d

ax_s.legend(plots, [p.get_label() for p in plots], loc=0)
plt.show()